In [ ]:
import os
import pathlib
from pathlib import Path
import glob
import re
import cx_Oracle as oracle
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
import openpyxl

##### CSVファイルのヘッダ行を取得する関数

In [2]:
def get_header(csv_file=None):
    with open(csv_file) as f:
        for row, text in enumerate(f, start=0):
            text = text.rstrip()
            if text == r'Data_Start':
                dst_row = row
                break
        else:
            raise ValueError('Not Found')
        
    return dst_row + 1

##### 必要なCSVファイルのパス＆ファイル名を取得する関数

In [3]:
def make_file_path_list():
    

    # カレントディレクトリを取得
    cwd = os.getcwd()

    # カレントディレクトリにある対象のデータフォルダ名を取得(複数ある場合に対応)
    target_folders = []
    for f in glob.glob(cwd + "\\*"):
        folder_name = os.path.split(f)[1]
        if re.findall("^\d{8}", folder_name):
            target_folders.append(folder_name)

    
    # 対象ディレクトリのパスを作成(list)
    path_list = [cwd + "\\" + target_folder + "\\*\\**" for target_folder in target_folders]
    
    # データフォルダ名のリストも作成
    folder_name_list = [target_folder for target_folder in target_folders]


    # 対象ディレクトリから全てのファイル名を取得(list)
    data_folders = []

    for path in path_list:
        data_folders.append(glob.glob(path))


    # 各データフォルダから必要なファイルを抽出し、各チャネル号機毎に5つのファイルを含んだリストを作成する
    # カレントフォルダにあるデータフォルダの個数分の長さのリストができる
    csv_lists = []
    for data_folder in data_folders:

        file_list=[]

        # ファイルリストから必要なファイルのみをフィルタリング
        for file in data_folder:
            file_list += [file]

        #各チャンネル毎で必要なcsvファイルのパスを取得し、リスト化
        CH_NO = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14']
        csv_list = []
        for ch in CH_NO:
            # JH放電容量・JT放電容量・JT-10℃放電抵抗に使用
            a = ch + '_CycleEnd'
            # JH25℃SOC50% 放電抵抗に使用
            b = ch + '_Count000005Ptn005'
            # JH-35℃SOC50% 放電抵抗に使用
            c = ch + '_Count000006Ptn006'
            # JH25℃SOC10% 放電抵抗に使用
            d = ch + '_Count000009Ptn009'
            # JM0℃SOC70% 充電抵抗に使用
            e = ch + '_Count000012Ptn012'
            # JT25℃SOC60% 放電抵抗に使用
            f = ch + '_Count000017Ptn017'
            # JT-30℃SOC15% 放電抵抗に使用
            g = ch + '_Count000020Ptn020'
            # JT-10℃SOC15% 放電抵抗に使用
            h = ch + '_Count000022Ptn022'

            i = [file for file in file_list if ('Grp' in file) and ( a in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( b in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( c in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( d in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( e in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( f in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( g in file) ]
            i += [file for file in file_list if ('Grp' in file) and ( h in file) ]

            csv_list.append(i)

        # 各データフォルダの最初のチャネル番号を取得してcsv_listにセットで格納
        for idx, ch in enumerate(csv_list, start=1):
            if ch:
                csv_list.insert(0, idx)
                break        
        
        csv_lists.append(csv_list)
    
    csv_lists_with_folder_name = []
    for folder_name, csv_list in zip(folder_name_list, csv_lists):
        csv_lists_with_folder_name.append((folder_name, csv_list))
        
    return csv_lists_with_folder_name
    

In [4]:
def get_index(df=None, cutoff_voltage=None, mode=1):
   
    """
    mode: 1 -> 以上
          2 -> 以下
    """
    # カットオフ電圧到達点があるかどうかをチェック
    df_original = df.copy()
    
    if mode == 1:
        df = df[df["Volt[V]"] >= cutoff_voltage]
    elif mode == 2:
        df = df[df["Volt[V]"] <= cutoff_voltage]

    # カット電圧到達点がある場合
    if len(df):
        # df.index.stop はそのDFの最終行番号のこと
        before_index = df_original.index.stop - len(df) - 1
        after_index = df_original.index.stop - len(df)
        return True, len(df)
    else:
        return False, 0

# CycleEnd-000k.csv を使用する項目 

### JH 放電容量を取得する関数

In [5]:
def get_jh_discharge_capacity(cycleEnd_filePath=None):

    # CycleEnd-000k.csvをDFに読み込む
    df_jh = pd.read_csv(cycleEnd_filePath, encoding="shift_jis", header = get_header(cycleEnd_filePath))

    # PtnNo. = 002:UF26TA_JH容量_25A充電CC,CC-CVでフィルタリング
    df_jh = df_jh.loc[df_jh["PtnNo."] == "002:UF26TA_JH容量_25A充電CC,CC-CV"]

    # Mode = 006:定電流放電(DC)又は007:定電圧放電(DV)でフィルタリング
    df_jh = df_jh[df_jh["Mode"].isin(["006:定電流放電(DC)","007:定電圧放電(DV)"])]

    # 上記フィルタリングを行うとデータが２行に絞れる。その2行のEndCapA[Ah]列の合計が放電容量
    jh_discahrge_capacity = sum(df_jh.loc[:,"EndCapA[Ah]"])
    
    # DB登録用のDFを準備
    df_jh_discahrge_capacity_for_db = df_jh[["EndCapA[Ah]", "PtnNo.", "EndTime[sec]", "EndCurr[A]", "EndVolt[V]", "EndTemp1[℃]", "InitVolt[V]"]].copy()
    
    # 検査項目の列を追加
    df_jh_discahrge_capacity_for_db["Item"] = "JH放電容量"
    
    return jh_discahrge_capacity, df_jh_discahrge_capacity_for_db

### JT 放電容量を取得する関数

In [6]:
def get_jt_discharge_capacity(cycleEnd_filePath=None):
    
    # CycleEnd-000k.csvをDFに読み込む
    df_jt = pd.read_csv(cycleEnd_filePath, encoding="shift_jis", header = get_header(cycleEnd_filePath))

    # PtnNo. = 003:UF26TA_JT容量_50A充電CC-CVでフィルタリング
    df_jt = df_jt.loc[df_jt["PtnNo."] == "003:UF26TA_JT容量_50A充電CC-CV"]

    # Mode = 007:定電圧放電(DV)でフィルタリング
    df_jt = df_jt[(df_jt["Mode"] == "007:定電圧放電(DV)")]

    # 上記フィルタリングを行うとデータが２行に絞れる。その2行目のEndCapA[Ah]列(11列目)の値がJT放電容量
    jt_discahrge_capacity = df_jt.iat[1, 11]
    
   # DB登録用のDFを準備
    df_jt_discahrge_capacity_for_db = df_jt[["EndCapA[Ah]", "PtnNo.", "EndTime[sec]", "EndCurr[A]", "EndVolt[V]", "EndTemp1[℃]", "InitVolt[V]"]].copy()
    
    # 検査項目の列を追加
    df_jt_discahrge_capacity_for_db["Item"] = "JT放電容量"
    
    return jt_discahrge_capacity, df_jt_discahrge_capacity_for_db

### JT -10℃SOC15% 5sec 充電抵抗を計算する関数

In [7]:

def calc_jt_n10_soc15_5sec_cr(cycleEnd_filePath=None):

    # ----------DF作成部分-------------
    
    # 変数名内のcr = 充電抵抗
    df_jt_cr_n10_soc15_5sec = pd.read_csv(cycleEnd_filePath, encoding="shift_jis", 
                                                            header = get_header(cycleEnd_filePath))

    # PtnNo. = 022:JT_SOC15%_-10℃ IVでフィルタリング
    df_jt_cr_n10_soc15_5sec = df_jt_cr_n10_soc15_5sec.loc[df_jt_cr_n10_soc15_5sec["PtnNo."] == "022:JT_SOC15%_-10℃ IV"]

    # Mode = 001:定電流充電(CC)でフィルタリング
    df_jt_cr_n10_soc15_5sec = df_jt_cr_n10_soc15_5sec[(df_jt_cr_n10_soc15_5sec["Mode"] == "001:定電流充電(CC)")]

    # EndTime[sec] = 5
    df_jt_cr_n10_soc15_5sec = df_jt_cr_n10_soc15_5sec[(df_jt_cr_n10_soc15_5sec["EndTime[sec]"] == 5)]

    # インデックス張り直し
    df_jt_cr_n10_soc15_5sec = df_jt_cr_n10_soc15_5sec.reset_index(drop = True)

    # 列の絞り込み前に開始電圧の値を取得しておく(15列目のInitVolt[V])
    init_volt = df_jt_cr_n10_soc15_5sec.iat[0, 15]

    # DB用にDFを別途作成
    df_jt_cr_n10_soc15_5sec_for_db = df_jt_cr_n10_soc15_5sec[["EndTime[sec]", "PtnNo.", "EndCurr[A]", "EndVolt[V]", "InitTemp1[℃]", "InitVolt[V]"]].copy()

    # 検査項目の列を追加
    df_jt_cr_n10_soc15_5sec_for_db["Item"] = "JT充電抵抗-10℃SOC15%5sec"
  
    # 必要な列のみを抽出
    df_jt_cr_n10_soc15_5sec = df_jt_cr_n10_soc15_5sec[["EndTime[sec]", "EndCurr[A]", "EndVolt[V]", "InitTemp1[℃]"]]
    
  
    
    # ----------計算部分-------------
    
    # まずは最終データポイント(pt8)の温度で条件分岐(-9℃)
    temp = df_jt_cr_n10_soc15_5sec.iat[7,3]

    # temp < -9 の場合は最終データポイント(pt8)を使用
    if temp < -9:
        # 到達電圧
        volt = df_jt_cr_n10_soc15_5sec.iat[7, 2]
        # 電流
        current = df_jt_cr_n10_soc15_5sec.iat[7, 1]

    # そうでない場合はpt7のデータを使用する
    else:
        # 到達電圧
        volt = df_jt_cr_n10_soc15_5sec.iat[6, 2]
        # 電流
        current = df_jt_cr_n10_soc15_5sec.iat[6, 1]

    # (到達電圧-開始電圧) / 電流 * 1000
    charging_resistance = (volt - init_volt) / current * 1000
    
    return charging_resistance, df_jt_cr_n10_soc15_5sec_for_db

# JT項目の処理

### JT 25℃SOC60% 5/0.2sec 放電抵抗を計算する関数

In [8]:
def calc_jt_25_soc60_dr(ptn17_jt_25_soc60_filePath=None):
    
    # ----------DF作成部分-------------

    # 変数名内のdr = 放電抵抗
    df_jt_dr_25_soc60 = pd.read_csv(ptn17_jt_25_soc60_filePath, encoding="shift_jis", 
                                                            header = get_header(ptn17_jt_25_soc60_filePath))

    # Mode = 6:定電流放電(DC)でフィルタリング
    df_jt_dr_25_soc60 = df_jt_dr_25_soc60[(df_jt_dr_25_soc60["Mode"] == "6:定電流放電(DC)")]

    # RngA = 1でフィルタリング
    df_jt_dr_25_soc60 = df_jt_dr_25_soc60[(df_jt_dr_25_soc60["RngA"] == 1)]

    # StpTime[sec] = 0 or 5 でフィルタリング (5sec用DF)
    df_jt_dr_25_soc60_5sec = df_jt_dr_25_soc60[(df_jt_dr_25_soc60["StpTime[sec]"] == 0) | (df_jt_dr_25_soc60["StpTime[sec]"] == 5)]

    # StpTime[sec] = 0 or 0.2 でフィルタリング (0.2sec用DF)
    df_jt_dr_25_soc60_02sec = df_jt_dr_25_soc60[(df_jt_dr_25_soc60["StpTime[sec]"] == 0) | (df_jt_dr_25_soc60["StpTime[sec]"] == 0.2)]


    # インデックス張り直し (ここまでのフィルタリングで16行のデータに絞られる)
    df_jt_dr_25_soc60_5sec = df_jt_dr_25_soc60_5sec.reset_index(drop = True)
    df_jt_dr_25_soc60_02sec = df_jt_dr_25_soc60_02sec.reset_index(drop = True)

    # DB用に必要な列のみに絞る
    df_jt_dr_25_soc60_5sec = df_jt_dr_25_soc60_5sec[["StpTime[sec]", "Curr[A]", "Volt[V]", "Temp1[℃]"]].copy()
    df_jt_dr_25_soc60_02sec = df_jt_dr_25_soc60_02sec[["StpTime[sec]", "Curr[A]", "Volt[V]", "Temp1[℃]"]].copy()
    
    
    # ----------計算部分-------------
    
    dr_5sec = calc_jt_dr_25_soc60(df_jt_dr_25_soc60_5sec)
    dr_02sec = calc_jt_dr_25_soc60(df_jt_dr_25_soc60_02sec)
    
    
    # DB用に検査項目名を追加
    df_jt_dr_25_soc60_5sec["Item"] = "JT 放電抵抗25℃SOC60%5sec" 
    df_jt_dr_25_soc60_02sec["Item"] = "JT 放電抵抗25℃SOC60%0.2sec" 

    
    return dr_5sec, dr_02sec, df_jt_dr_25_soc60_5sec, df_jt_dr_25_soc60_02sec


# JT 25℃SOC60%　の放電抵抗を計算する関数
# データポイントを取ってきて計算する部分のみ
def calc_jt_dr_25_soc60(df=None):
    
    # StepNo.57のデータ(DFの最後２行)を使用して抵抗値を算出
    # 1列目:Curr[A], 2列目:Volt[V]
    current_0 = df.iat[14, 1]
    current_5 = df.iat[15, 1]
    volt_0 = df.iat[14, 2]
    volt_5 = df.iat[15, 2]
    
    # 抵抗値の算出
    return (volt_0 - volt_5) / (current_0 - current_5) * 1000

### JT -30℃SOC15% 2/0.2sec 放電抵抗を計算する関数

In [9]:
def calc_jt_n30_soc15_dr(ptn20_jt_n30_soc15_filePath=None):

    # ----------DF作成部分-------------
    
    # 変数名内のdr = 放電抵抗
    df_jt_dr_n30_soc15 = pd.read_csv(ptn20_jt_n30_soc15_filePath, encoding="shift_jis", 
                                                            header = get_header(ptn20_jt_n30_soc15_filePath))

    # Mode = 6:定電流放電(DC)でフィルタリング
    df_jt_dr_n30_soc15 = df_jt_dr_n30_soc15[(df_jt_dr_n30_soc15["Mode"] == "6:定電流放電(DC)")]

    # RngA = 1 or 2 でフィルタリング
    df_jt_dr_n30_soc15 = df_jt_dr_n30_soc15[(df_jt_dr_n30_soc15["RngA"] == 1) | (df_jt_dr_n30_soc15["RngA"] == 2)]

    # StpTime[sec] = 0 or 2 でフィルタリング (2sec用DF)
    df_jt_dr_n30_soc15_2sec = df_jt_dr_n30_soc15[(df_jt_dr_n30_soc15["StpTime[sec]"] == 0) | (df_jt_dr_n30_soc15["StpTime[sec]"] == 2)]

    # StpTime[sec] = 0 or 0.2 でフィルタリング (0.2sec用DF)
    df_jt_dr_n30_soc15_02sec = df_jt_dr_n30_soc15[(df_jt_dr_n30_soc15["StpTime[sec]"] == 0) | (df_jt_dr_n30_soc15["StpTime[sec]"] == 0.2)]


    # インデックス張り直し (ここまでのフィルタリングで14行のデータに絞られる)
    df_jt_dr_n30_soc15_2sec_original = df_jt_dr_n30_soc15_2sec.reset_index(drop = True)
    # インデックス張り直し (ここまでのフィルタリングで16行のデータに絞られる)
    df_jt_dr_n30_soc15_02sec_original = df_jt_dr_n30_soc15_02sec.reset_index(drop = True)

    # DB用に必要な列のみに絞る
    df_jt_dr_n30_soc15_2sec_col_filtered = df_jt_dr_n30_soc15_2sec_original[["StpTime[sec]", "Curr[A]", "Volt[V]", "Temp1[℃]"]]
    df_jt_dr_n30_soc15_02sec_col_filtered = df_jt_dr_n30_soc15_02sec_original[["StpTime[sec]", "Curr[A]", "Volt[V]", "Temp1[℃]"]]

    # さらに計算に必要な行のみに絞る
    df_jt_dr_n30_soc15_2sec_row_filtered = df_jt_dr_n30_soc15_2sec_col_filtered.iloc[[0,1,3,5,7,9,11,13],:].reset_index(drop = True).copy()
    df_jt_dr_n30_soc15_02sec_row_filtered = df_jt_dr_n30_soc15_02sec_col_filtered.iloc[[0,1,3,5,7,9,11,13,15],:].reset_index(drop = True).copy()
    
    
    # カット電圧到達点の有無をチェックして使用するデータポイントを決める
    w_or_wo, num = get_index(df_jt_dr_n30_soc15_2sec_row_filtered, cutoff_voltage=2, mode=2)

    # カット電圧到達点がある場合、到達前後の2点のデータを使用する
    if w_or_wo:
        df_jt_dr_n30_soc15_2sec_calc = df_jt_dr_n30_soc15_2sec_row_filtered.iloc[[-(num + 2), -(num + 1)], :]

    # カット電圧到達点がない場合、5, 6行目のデータを使用する
    else:
        df_jt_dr_n30_soc15_2sec_calc = df_jt_dr_n30_soc15_2sec_row_filtered.iloc[[5, 6], :]
        
        
    # ----------計算部分(2sec)-------------

    # scikit-learn用にフォーマットを整える
    x = df_jt_dr_n30_soc15_2sec_calc[['Volt[V]']]
    # 電流値の符号反転
    y = -(df_jt_dr_n30_soc15_2sec_calc[['Curr[A]']])

    # scikit-learnのを使って単回帰モデルの作成
    model = LinearRegression()
    model.fit(x, y)

    # 作成した回帰モデルを使用して、カット電圧値における電流値を算出
    current = model.coef_[0][0] * 2 + model.intercept_[0]

    # StepNo.41, 電流値0Aの時の電圧値を到達電圧として決め打ちで使用(理由は不明)
    volt = df_jt_dr_n30_soc15_2sec_col_filtered.iat[10, 2]

    # (到達電圧 - カット電圧) / 電流値
    dr_2sec = (volt - 2) / current * 1000
    
    
    # ----------計算部分(0.2sec)-------------
    
    # 0.2secの方は使用するデータポイントは決め打ち
    # StepNo.41, 電流値0Aの時の電圧値と電流値-120Aの時の電圧値
    volt1 = df_jt_dr_n30_soc15_02sec_col_filtered.iat[10, 2]
    volt2 = df_jt_dr_n30_soc15_02sec_col_filtered.iat[11, 2]

    # 電流値は、StepNo.41の電流値(-120A)
    current = -(df_jt_dr_n30_soc15_02sec_col_filtered.iat[11, 1])

    # 抵抗値の算出
    dr_02sec = (volt1 - volt2) / current * 1000
    
    
    # DB用に検査項目列を追加
    df_jt_dr_n30_soc15_2sec_row_filtered["Item"] = "JT 放電抵抗-30℃SOC15%2sec"
    df_jt_dr_n30_soc15_02sec_row_filtered["Item"] = "JT 放電抵抗-30℃SOC15%0.2sec"

    return dr_2sec, dr_02sec, df_jt_dr_n30_soc15_2sec_row_filtered, df_jt_dr_n30_soc15_02sec_row_filtered

### JT -10℃SOC15% 5/0.2sec 放電抵抗を計算する関数

In [10]:
def calc_jt_n10_soc15_dr(ptn22_jt_n10_soc15_filePath=None):
    
    # ----------DF作成部分-------------
    
    # 変数名内のdr = 放電抵抗
    df_jt_dr_n10_soc15 = pd.read_csv(ptn22_jt_n10_soc15_filePath, encoding="shift_jis", 
                                                            header = get_header(ptn22_jt_n10_soc15_filePath))

    # Mode = 6:定電流放電(DC)でフィルタリング
    df_jt_dr_n10_soc15 = df_jt_dr_n10_soc15[(df_jt_dr_n10_soc15["Mode"] == "6:定電流放電(DC)")]

    # RngA = 1 でフィルタリング
    df_jt_dr_n10_soc15 = df_jt_dr_n10_soc15[(df_jt_dr_n10_soc15["RngA"] == 1)]

    # StpTime[sec] = 0 or 5 でフィルタリング (5sec用DF)
    df_jt_dr_n10_soc15_5sec = df_jt_dr_n10_soc15[(df_jt_dr_n10_soc15["StpTime[sec]"] == 0) | (df_jt_dr_n10_soc15["StpTime[sec]"] == 5)]

    # StpTime[sec] = 0 or 0.2 でフィルタリング (0.2sec用DF)
    df_jt_dr_n10_soc15_02sec = df_jt_dr_n10_soc15[(df_jt_dr_n10_soc15["StpTime[sec]"] == 0) | (df_jt_dr_n10_soc15["StpTime[sec]"] == 0.2)]


    # インデックス張り直し (ここまでのフィルタリングで13行のデータに絞られる)
    df_jt_dr_n10_soc15_5sec_original = df_jt_dr_n10_soc15_5sec.reset_index(drop = True)
    # インデックス張り直し (ここまでのフィルタリングで16行のデータに絞られる)
    df_jt_dr_n10_soc15_02sec_original = df_jt_dr_n10_soc15_02sec.reset_index(drop = True)

    # DB用に必要な列のみに絞る
    df_jt_dr_n10_soc15_5sec_col_filtered = df_jt_dr_n10_soc15_5sec_original[["StpTime[sec]", "Curr[A]", "Volt[V]", "Temp1[℃]"]].copy()
    df_jt_dr_n10_soc15_02sec_col_filtered = df_jt_dr_n10_soc15_02sec_original[["StpTime[sec]", "Curr[A]", "Volt[V]", "Temp1[℃]"]].copy()

    # さらに計算に必要な行のみに絞る
    df_jt_dr_n10_soc15_5sec_row_filtered = df_jt_dr_n10_soc15_5sec_col_filtered.iloc[[0,1,3,5,7,9,11],:].reset_index(drop = True)
    df_jt_dr_n10_soc15_02sec_row_filtered = df_jt_dr_n10_soc15_02sec_col_filtered.iloc[[0,1,3,5,7,9,11,13,15],:].reset_index(drop = True)
    
    
    # ----------計算部分(5sec)-------------
    
    # カット電圧到達点の有無をチェックして使用するデータポイントを決める
    w_or_wo, num = get_index(df_jt_dr_n10_soc15_5sec_row_filtered, cutoff_voltage=2.5, mode=2)

    # カット電圧到達点がある場合、到達前後の2点のデータを使用する
    if w_or_wo:
        df_jt_dr_n10_soc15_5sec_calc = df_jt_dr_n10_soc15_5sec_row_filtered.iloc[[-(num + 1), -(num + 2)], :]

    # カット電圧到達点がない場合、4, 5行目のデータを使用する
    else:
        df_jt_dr_n10_soc15_5sec_calc = df_jt_dr_n10_soc15_5sec_row_filtered.iloc[[4, 5], :]


    # scikit-learn用にフォーマットを整える
    x = df_jt_dr_n10_soc15_5sec_calc[['Volt[V]']]
    # 電流値の符号反転
    y = -(df_jt_dr_n10_soc15_5sec_calc[['Curr[A]']])

    # scikit-learnのを使って単回帰モデルの作成
    model = LinearRegression()
    model.fit(x, y)

    # 作成した回帰モデルを使用して、カット電圧値における電流値を算出
    current = model.coef_[0][0] * 2.5 + model.intercept_[0]

    # 電圧値は、3.4987で固定
    volt = 3.4987

    # (到達電圧 - カット電圧) / 電流値
    dr_5sec = (volt - 2.5) / current * 1000
    
    
    # ----------計算部分(0.2sec)-------------
    
    # カット電圧到達点の有無をチェックして使用するデータポイントを決める
    w_or_wo, num = get_index(df_jt_dr_n10_soc15_02sec_row_filtered, cutoff_voltage=2.5, mode=2)

    # カット電圧到達点がある場合、到達前後の2点のデータを使用する
    if w_or_wo:
        df_jt_dr_n10_soc15_02sec_calc = df_jt_dr_n10_soc15_02sec_row_filtered.iloc[[-(num + 1), -(num + 2)], :]

    # カット電圧到達点がない場合、7, 8行目のデータを使用する
    else:
        df_jt_dr_n10_soc15_02sec_calc = df_jt_dr_n10_soc15_02sec_row_filtered.iloc[[7, 8], :]



    # 電圧値は、StpNo.57 電流値0のときの電圧値と電流値-380[A]のときの電圧値を使用する
    volt1 = df_jt_dr_n10_soc15_02sec_col_filtered.iat[14, 2]
    volt2 = df_jt_dr_n10_soc15_02sec_row_filtered.iat[8, 2]

    # 電流値は、StpNo.57の電流値-380[A]を使用する(符号反転)
    current = -(df_jt_dr_n10_soc15_02sec_row_filtered.iat[8, 1])

    # (到達電圧 - カット電圧) / 電流値
    dr_02sec = (volt1 - volt2) / current * 1000
    
    
    # DB用に検査項目の列を追加
    df_jt_dr_n10_soc15_5sec_col_filtered["Item"] = "JT 放電抵抗-10℃SOC15%5sec"
    df_jt_dr_n10_soc15_02sec_col_filtered["Item"] = "JT 放電抵抗-10℃SOC15%0.2sec"

    
    return dr_5sec, dr_02sec, df_jt_dr_n10_soc15_5sec_col_filtered, df_jt_dr_n10_soc15_02sec_col_filtered

# JH/JM 項目の処理

### JH 25℃SOC50% 10sec の放電抵抗＆放電出力を計算する関数

In [11]:
def calc_jh_25_soc50_10sec(ptn05_jh_25_soc50_filePath=None):
    
    # ----------DF作成部分-------------
    
    # 変数名内のdr = 放電抵抗
    df_jh_dr_25_soc50 = pd.read_csv(ptn05_jh_25_soc50_filePath, encoding="shift_jis", 
                                                            header = get_header(ptn05_jh_25_soc50_filePath))

    # Mode = 6:定電流放電(DC)でフィルタリング
    df_jh_dr_25_soc50 = df_jh_dr_25_soc50[(df_jh_dr_25_soc50["Mode"] == "6:定電流放電(DC)")]

    # StpTime[sec] = 10 でフィルタリングしたものの上12行
    df_jh_dr_25_soc50_10sec = df_jh_dr_25_soc50[(df_jh_dr_25_soc50["StpTime[sec]"] == 10)].iloc[:12, :]

    # インデックス張り直し 
    df_jh_dr_25_soc50_10sec = df_jh_dr_25_soc50_10sec.reset_index(drop = True)

    # DB用に必要な列のみに絞る
    df_jh_dr_25_soc50_10sec_col_filtered = df_jh_dr_25_soc50_10sec[["StpTime[sec]", "Curr[A]", "Volt[V]", "Temp1[℃]"]].copy()
    
    # カット電圧到達点の有無をチェックして使用するデータポイントを決める
    w_or_wo, num = get_index(df_jh_dr_25_soc50_10sec_col_filtered, cutoff_voltage=2.5, mode=2)

    # カット電圧到達点がある場合、到達前までのデータを使用する
    if w_or_wo:
        df_jh_dr_25_soc50_10sec_calc = df_jh_dr_25_soc50_10sec_col_filtered.iloc[0:-num, :]

    # カット電圧到達点がない場合、全データを使用する
    else:
        df_jh_dr_25_soc50_10sec_calc = df_jh_dr_25_soc50_10sec_col_filtered.iloc[:, :]
        
        
    # ----------計算部分(放電抵抗)-------------
    
    # scikit-learn用にフォーマットを整える
    x = df_jh_dr_25_soc50_10sec_calc[['Volt[V]']]
    # 電流値の符号反転
    y = -(df_jh_dr_25_soc50_10sec_calc[['Curr[A]']])

    # scikit-learnのを使って単回帰モデルの作成
    model = LinearRegression()
    model.fit(y, x)

    # 傾き(抵抗)を算出
    coef = model.coef_[0][0]
    resistance = -(coef * 1000)
    
    
    # ----------計算部分(放電出力)-------------
    
    # まずは2次近似の各値を算出

    # scikit-learn用にフォーマットを整える
    x = df_jh_dr_25_soc50_10sec_calc[['Volt[V]']]
    # 電流値の符号反転
    y = -(df_jh_dr_25_soc50_10sec_calc[['Curr[A]']])

    # 説明変数のデータを２次式用に加工
    polynomial_features = PolynomialFeatures(degree=2)
    x_poly = polynomial_features.fit_transform(x)

    # y = b0 + b1x + b2x^2 の b0～b2 を算出
    model = LinearRegression()
    model.fit(x_poly, y)
    y_pred = model.predict(x_poly)

    # 各係数および定数項を算出
    coefficient1, coefficient2, const = model.coef_[0][1], model.coef_[0][2], model.intercept_[0]

    # 回帰式を用いてカット電圧値における電流値を求める
    current = coefficient2 * (2.5 ** 2) + coefficient1 * 2.5 + const

    # カット電圧と上で近似式を使って算出した電流値を使って出力(W)を計算
    w = current * 2.5
    

    # DB用に検査項目名を追加
    df_jh_dr_25_soc50_10sec_col_filtered["Item"] = "JH 放電出力・放電抵抗25℃SOC50%10sec"

    
    return resistance, w, df_jh_dr_25_soc50_10sec_col_filtered

### JH -35℃SOC50% 10sec の放電抵抗＆放電出力を計算する関数

In [12]:
def calc_jh_n35_soc50_10sec(ptn06_jh_n35_soc50_filePath=None):

    # ----------DF作成部分-------------
    
    # 変数名内のdr = 放電抵抗
    df_jh_dr_n35_soc50 = pd.read_csv(ptn06_jh_n35_soc50_filePath, encoding="shift_jis", 
                                                            header = get_header(ptn06_jh_n35_soc50_filePath))

    # Mode = 6:定電流放電(DC)でフィルタリング
    df_jh_dr_n35_soc50 = df_jh_dr_n35_soc50[(df_jh_dr_n35_soc50["Mode"] == "6:定電流放電(DC)")]

    # StpTime[sec] = 10 でフィルタリングしたものの上12行
    df_jh_dr_n35_soc50_10sec = df_jh_dr_n35_soc50[(df_jh_dr_n35_soc50["StpTime[sec]"] == 10)].iloc[:12, :]

    # インデックス張り直し 
    df_jh_dr_n35_soc50_10sec = df_jh_dr_n35_soc50_10sec.reset_index(drop = True)

    # DB用に必要な列のみに絞る
    df_jh_dr_n35_soc50_10sec_col_filtered = df_jh_dr_n35_soc50_10sec[["StpTime[sec]", "Curr[A]", "Volt[V]", "Temp1[℃]"]].copy()
    
    # カット電圧到達点の有無をチェックして使用するデータポイントを決める
    w_or_wo, num = get_index(df_jh_dr_n35_soc50_10sec_col_filtered, cutoff_voltage=2.5, mode=2)

    # カット電圧到達点がある場合、到達前までのデータを使用する
    if w_or_wo:
        df_jh_dr_n35_soc50_10sec_calc = df_jh_dr_n35_soc50_10sec_col_filtered.iloc[0:-num, :]

    # カット電圧到達点がない場合、全データを使用する
    else:
        df_jh_dr_n35_soc50_10sec_calc = df_jh_dr_n35_soc50_10sec_col_filtered.iloc[:, :]
    

    # ----------計算部分(放電抵抗)-------------
    
    # scikit-learn用にフォーマットを整える
    x = df_jh_dr_n35_soc50_10sec_calc[['Volt[V]']]
    # 電流値の符号反転
    y = -(df_jh_dr_n35_soc50_10sec_calc[['Curr[A]']])

    # scikit-learnのを使って単回帰モデルの作成
    model = LinearRegression()
    model.fit(y, x)

    # 傾き(抵抗)を算出
    coef = model.coef_[0][0]
    resistance = -(coef * 1000)
    
    
    # ----------計算部分(放電出力)-------------
    
    # まずは2次近似の各値を算出

    # scikit-learn用にフォーマットを整える
    x = df_jh_dr_n35_soc50_10sec_calc[['Volt[V]']]
    # 電流値の符号反転
    y = -(df_jh_dr_n35_soc50_10sec_calc[['Curr[A]']])

    # 説明変数のデータを２次式用に加工
    polynomial_features = PolynomialFeatures(degree=2)
    x_poly = polynomial_features.fit_transform(x)

    # y = b0 + b1x + b2x^2 の b0～b2 を算出
    model = LinearRegression()
    model.fit(x_poly, y)
    y_pred = model.predict(x_poly)

    # 各係数および定数項を算出
    coefficient1, coefficient2, const = model.coef_[0][1], model.coef_[0][2], model.intercept_[0]

    # 回帰式を用いてカット電圧値における電流値を求める
    current = coefficient2 * (2.5 ** 2) + coefficient1 * 2.5 + const

    # カット電圧と上で近似式を使って算出した電流値を使って出力(W)を計算
    w = current * 2.5
    
    
    # DB用に必要な列のみに絞る
    df_jh_dr_n35_soc50_10sec_col_filtered["Item"] = "JH 放電出力・放電抵抗-35℃SOC50%10sec"

    return resistance, w, df_jh_dr_n35_soc50_10sec_col_filtered

### JH 25℃SOC10% 10sec の放電抵抗＆放電出力を計算する関数

In [13]:
def calc_jh_25_soc10_10sec(ptn09_jh_25_soc10_filePath=None):
    
    # ----------DF作成部分-------------
    
    # 変数名内のdr = 放電抵抗
    df_jh_dr_25_soc10 = pd.read_csv(ptn09_jh_25_soc10_filePath, encoding="shift_jis", 
                                                            header = get_header(ptn09_jh_25_soc10_filePath))

    # Mode = 6:定電流放電(DC)でフィルタリング
    df_jh_dr_25_soc10 = df_jh_dr_25_soc10[(df_jh_dr_25_soc10["Mode"] == "6:定電流放電(DC)")]

    # StpTime[sec] = 10 でフィルタリングしたものの上12行
    df_jh_dr_25_soc10_10sec = df_jh_dr_25_soc10[(df_jh_dr_25_soc10["StpTime[sec]"] == 10)].iloc[:12, :]

    # インデックス張り直し 
    df_jh_dr_25_soc10_10sec = df_jh_dr_25_soc10_10sec.reset_index(drop = True)

    # DB用に必要な列のみに絞る
    df_jh_dr_25_soc10_10sec_col_filtered = df_jh_dr_25_soc10_10sec[["StpTime[sec]", "Curr[A]", "Volt[V]", "Temp1[℃]"]].copy()

    # この項目はカット電圧の有無に関係なく全テータポイントを使用する
    df_jh_dr_25_soc10_10sec_calc = df_jh_dr_25_soc10_10sec_col_filtered.copy()
    
    
    # ----------計算部分(放電抵抗)-------------
    
    # scikit-learn用にフォーマットを整える
    x = df_jh_dr_25_soc10_10sec_calc[['Volt[V]']]
    # 電流値の符号反転
    y = -(df_jh_dr_25_soc10_10sec_calc[['Curr[A]']])

    # scikit-learnのを使って単回帰モデルの作成
    model = LinearRegression()
    model.fit(y, x)

    # 傾き(抵抗)を算出
    coef = model.coef_[0][0]
    resistance = -(coef * 1000)
    
    
    # ----------計算部分(放電出力)-------------
    
    # まずは2次近似の各値を算出

    # scikit-learn用にフォーマットを整える
    x = df_jh_dr_25_soc10_10sec_calc[['Volt[V]']]
    # 電流値の符号反転
    y = -(df_jh_dr_25_soc10_10sec_calc[['Curr[A]']])

    # 説明変数のデータを２次式用に加工
    polynomial_features = PolynomialFeatures(degree=2)
    x_poly = polynomial_features.fit_transform(x)

    # y = b0 + b1x + b2x^2 の b0～b2 を算出
    model = LinearRegression()
    model.fit(x_poly, y)
    y_pred = model.predict(x_poly)

    # 各係数および定数項を算出
    coefficient1, coefficient2, const = model.coef_[0][1], model.coef_[0][2], model.intercept_[0]

    # 回帰式を用いてカット電圧値における電流値を求める
    current = coefficient2 * (2.5 ** 2) + coefficient1 * 2.5 + const

    # カット電圧と上で近似式を使って算出した電流値を使って出力(W)を計算
    w = current * 2.5
    
    
    # DB用に必要な列のみに絞る
    df_jh_dr_25_soc10_10sec_col_filtered["Item"] = "JH 放電出力・放電抵抗25℃SOC10%10sec"

    
    return resistance, w, df_jh_dr_25_soc10_10sec_col_filtered

### JM 0℃SOC70% 10sec の充電抵抗＆充電出力を計算する関数

In [14]:
def calc_jm_0_soc70_10sec(ptn12_jm_0_soc70_filePath=None):

    # ----------DF作成部分-------------
    
    # 変数名内のdr = 放電抵抗
    df_jm_dr_0_soc70 = pd.read_csv(ptn12_jm_0_soc70_filePath, encoding="shift_jis", 
                                                            header = get_header(ptn12_jm_0_soc70_filePath))

    # Mode = 1:定電流充電(CC) or 19:ステップ終了でフィルタリング
    df_jm_dr_0_soc70 = df_jm_dr_0_soc70[(df_jm_dr_0_soc70["Mode"] == "1:定電流充電(CC)") | (df_jm_dr_0_soc70["Mode"] == "19:ステップ終了")]

    # StpTime[sec] = 10 でフィルタリング(全部で7行に絞られる)
    df_jm_dr_0_soc70_10sec = df_jm_dr_0_soc70[(df_jm_dr_0_soc70["StpTime[sec]"] == 10)]

    # インデックス張り直し 
    df_jm_dr_0_soc70_10sec = df_jm_dr_0_soc70_10sec.reset_index(drop = True)

    # DB用に必要な列のみに絞る
    df_jm_dr_0_soc70_10sec_col_filtered = df_jm_dr_0_soc70_10sec[["StpTime[sec]", "Curr[A]", "Volt[V]", "Temp1[℃]"]].copy()
    
    # カット電圧到達点の有無をチェックして使用するデータポイントを決める
    w_or_wo, num = get_index(df_jm_dr_0_soc70_10sec_col_filtered, cutoff_voltage=4.3, mode=1)

    # カット電圧到達点がある場合、到達前までのデータを使用する
    if w_or_wo:
        df_jm_dr_0_soc70_10sec_calc = df_jm_dr_0_soc70_10sec_col_filtered.iloc[0:-num, :]

    # カット電圧到達点がない場合、全データを使用する
    else:
        df_jm_dr_0_soc70_10sec_calc = df_jm_dr_0_soc70_10sec_col_filtered.iloc[:, :]
        
        
    # ----------計算部分(充電抵抗)-------------
    
    # scikit-learn用にフォーマットを整える
    x = df_jm_dr_0_soc70_10sec_calc[['Volt[V]']]
    # 電流値の符号反転
    y = df_jm_dr_0_soc70_10sec_calc[['Curr[A]']]

    # scikit-learnのを使って単回帰モデルの作成
    model = LinearRegression()
    model.fit(y, x)

    # 傾き(抵抗)を算出
    coef = model.coef_[0][0]
    resistance = coef * 1000
    
    
    # ----------計算部分(充電出力)-------------
    
    # まずは2次近似の各値を算出

    # scikit-learn用にフォーマットを整える
    x = df_jm_dr_0_soc70_10sec_calc[['Volt[V]']]
    # 電流値の符号反転
    y = df_jm_dr_0_soc70_10sec_calc[['Curr[A]']]

    # 説明変数のデータを２次式用に加工
    polynomial_features = PolynomialFeatures(degree=2)
    x_poly = polynomial_features.fit_transform(x)

    # y = b0 + b1x + b2x^2 の b0～b2 を算出
    model = LinearRegression()
    model.fit(x_poly, y)
    y_pred = model.predict(x_poly)

    # 各係数および定数項を算出
    coefficient1, coefficient2, const = model.coef_[0][1], model.coef_[0][2], model.intercept_[0]

    # 回帰式を用いてカット電圧値における電流値を求める
    current = coefficient2 * (4.3 ** 2) + coefficient1 * 4.3 + const

    # カット電圧と上で近似式を使って算出した電流値を使って出力(W)を計算
    w = current * 4.3
    
    
    # DB用に必要な列のみに絞る
    df_jm_dr_0_soc70_10sec_col_filtered["Item"] = "JM 充電出力・充電抵抗0℃SOC70%10sec"

    
    return resistance, w, df_jm_dr_0_soc70_10sec_col_filtered

### 演算および演算前後のDFを返す関数

In [15]:
def make_result_list(csv_list=None, df_sample=None):
    
    
    # 全チャネルの演算結果を格納するためのリストを作成
    all_ch_result_list = []
    
    # 各検査項目ごとのDFを格納するリストを準備(計13個)
    
    # (8個)
    jt_discahrge_capacity_df_list = []
    jt_25_soc60_5sec_dr_df_list = []
    jt_25_soc60_02sec_dr_df_list = []
    jt_n10_soc15_5sec_cr_df_list = []
    jt_n30_soc15_2sec_dr_df_list = []
    jt_n30_soc15_02sec_dr_df_list = []
    jt_n10_soc15_5sec_dr_df_list = []
    jt_n10_soc15_02sec_dr_df_list = []
    
    # JH/JM (5個)
    jh_discharge_capacity_df_list = []
    jh_25_soc50_10sec_df_list = []
    jh_n35_soc50_10sec_df_list = []
    jh_25_soc10_10sec_df_list = []
    jm_0_soc70_10sec_df_list = []
    
    
    i = 0
    for ch_num, ch_folder in enumerate(csv_list, 1):
        
        if ch_folder:
            
            # 抜取DFの各列の値を変数にセット
            mod_type = df_sample.iat[i, 0]
            klot = df_sample.iat[i, 1]
            line_no = df_sample.iat[i, 2]
            cell_id = df_sample.iat[i, 3]
            machine_no = df_sample.iat[i, 4]
            ch_no = ch_num
            
            i += 1
            
            # 各チャネルのフォルダから必要なファイルパスを取り出して変数に設定
            cycleEnd_filePath = ch_folder[0]
            ptn05_jh_25_soc50_filePath = ch_folder[1]
            ptn06_jh_n35_soc50_filePath = ch_folder[2]
            ptn09_jh_25_soc10_filePath = ch_folder[3]
            ptn12_jm_0_soc70_filePath = ch_folder[4]
            ptn17_jt_25_soc60_filePath = ch_folder[5]
            ptn20_jt_n30_soc15_filePath = ch_folder[6]
            ptn22_jt_n10_soc15_filePath = ch_folder[7]
         
            # 日付を格納する変数を準備
            test_date = ""

            # 日付をcsvファイルから抽出
            with open(cycleEnd_filePath) as f:
                folder_name = ""
                for row, text in enumerate(f, start=0):
                    text = text.rstrip()
                    if "ﾃﾞｰﾀ保存ﾌｫﾙﾀﾞ" in text:
                        folder_name = text
                    if "担当者" in text:
                        person_name = text
                        
                    if (folder_name and person_name):
                        break
                        
            test_date = re.findall("\d{8}", folder_name)[0]
            person = re.findall('"(.*)"', person_name)[0]
            
            
            # JT各種演算と演算前DFの準備
            jt_discharge_capacity, df_jt_discahrge_capacity_for_db = get_jt_discharge_capacity(cycleEnd_filePath)
            jt_25_soc60_5sec_dr, jt25_soc60_02sec_dr, df_jt_dr_25_soc60_5sec_for_db, df_jt_dr_25_soc60_02sec_for_db = calc_jt_25_soc60_dr(ptn17_jt_25_soc60_filePath)
            jt_n10_soc15_5sec_cr, df_jt_cr_n10_soc15_5sec_for_db = calc_jt_n10_soc15_5sec_cr(cycleEnd_filePath)
            jt_n30_soc15_2sec_dr, jt_n30_soc15_02sec_dr, df_jt_dr_n30_soc15_2sec_for_db, df_jt_dr_n30_soc15_02sec_for_db = calc_jt_n30_soc15_dr(ptn20_jt_n30_soc15_filePath)
            jt_n10_soc15_5sec_dr, jt_n10_soc15_02sec_dr, df_jt_dr_n10_soc15_5sec_for_db, df_jt_dr_n10_soc15_02sec_for_db = calc_jt_n10_soc15_dr(ptn22_jt_n10_soc15_filePath)
            
            # JH/JM各種演算と演算前DFの準備
            jh_discharge_capacity, df_jh_discahrge_capacity_for_db = get_jh_discharge_capacity(cycleEnd_filePath)
            jh_25_soc50_10sec_dr, jh_25_soc50_10sec_w, df_jh_dr_25_soc50_10sec_for_db = calc_jh_25_soc50_10sec(ptn05_jh_25_soc50_filePath)
            jh_n35_soc50_10sec_dr, jh_n35_soc50_10sec_w, df_jh_dr_n35_soc50_10sec_for_db = calc_jh_n35_soc50_10sec(ptn06_jh_n35_soc50_filePath)
            jh_25_soc10_10sec_dr, jh_25_soc10_10sec_w, df_jh_dr_25_soc10_10sec_for_db = calc_jh_25_soc10_10sec(ptn09_jh_25_soc10_filePath)
            jm_0_soc70_10sec_dr, jm_0_soc70_10sec_w, df_jm_dr_0_soc70_10sec_for_db = calc_jm_0_soc70_10sec(ptn12_jm_0_soc70_filePath)
            
            # 検査項目毎のDFに抜取用DFのデータを追加
            
            before_df_list = [df_jt_discahrge_capacity_for_db, df_jt_dr_25_soc60_5sec_for_db, df_jt_dr_25_soc60_02sec_for_db,
                             df_jt_cr_n10_soc15_5sec_for_db, df_jt_dr_n30_soc15_2sec_for_db, df_jt_dr_n30_soc15_02sec_for_db,
                             df_jt_dr_n10_soc15_5sec_for_db, df_jt_dr_n10_soc15_02sec_for_db,
                             df_jh_discahrge_capacity_for_db, df_jh_dr_25_soc50_10sec_for_db, df_jh_dr_n35_soc50_10sec_for_db,
                             df_jh_dr_25_soc10_10sec_for_db, df_jm_dr_0_soc70_10sec_for_db]
            
            
            for before_df in before_df_list:
                
                before_df["TestDate"] = test_date
                before_df["LineNo"] = line_no
                before_df["ModType"] = mod_type
                before_df["Klot"] = klot
                before_df["MachineNo"] = machine_no
                before_df["ChNo"] = ch_no
                before_df["CellId"] = cell_id
                    
        
            # 各チャネルの演算前DFをリストに追加
            # JT
            jt_discahrge_capacity_df_list.append(df_jt_discahrge_capacity_for_db)
            jt_25_soc60_5sec_dr_df_list.append(df_jt_dr_25_soc60_5sec_for_db)
            jt_25_soc60_02sec_dr_df_list.append(df_jt_dr_25_soc60_02sec_for_db)
            jt_n10_soc15_5sec_cr_df_list.append(df_jt_cr_n10_soc15_5sec_for_db)
            jt_n30_soc15_2sec_dr_df_list.append(df_jt_dr_n30_soc15_2sec_for_db)
            jt_n30_soc15_02sec_dr_df_list.append(df_jt_dr_n30_soc15_02sec_for_db)
            jt_n10_soc15_5sec_dr_df_list.append(df_jt_dr_n10_soc15_5sec_for_db)
            jt_n10_soc15_02sec_dr_df_list.append(df_jt_dr_n10_soc15_02sec_for_db)

            # JH/JM 
            jh_discharge_capacity_df_list.append(df_jh_discahrge_capacity_for_db)
            jh_25_soc50_10sec_df_list.append(df_jh_dr_25_soc50_10sec_for_db)
            jh_n35_soc50_10sec_df_list.append(df_jh_dr_n35_soc50_10sec_for_db)
            jh_25_soc10_10sec_df_list.append(df_jh_dr_25_soc10_10sec_for_db)
            jm_0_soc70_10sec_df_list.append(df_jm_dr_0_soc70_10sec_for_db)
            
            
            # 演算結果をDBの列に合わせて格納しているだけ
            calc_result = [jt_discharge_capacity, jt_25_soc60_5sec_dr, jt_n30_soc15_2sec_dr, jt_n10_soc15_5sec_dr, \
                          jt_n10_soc15_5sec_cr, jt25_soc60_02sec_dr, jt_n30_soc15_02sec_dr, jt_n10_soc15_02sec_dr, \
                          jh_discharge_capacity, jh_25_soc50_10sec_w, jh_n35_soc50_10sec_w, jh_25_soc10_10sec_w, jm_0_soc70_10sec_w, \
                          jh_25_soc50_10sec_dr, jh_n35_soc50_10sec_dr, jh_25_soc10_10sec_dr, jm_0_soc70_10sec_dr]
            
            # calc_resultの先頭にチャネル番号を追加
            calc_result.insert(0, ch_num)
            

                
            # calc_resultの先頭に日付を追加
            calc_result.insert(0, test_date)            
            
            all_ch_result_list.append(calc_result)
    
    

    # 全チャネルの検査項目毎のDFをマージ
    # JT
    before_df_jt_discahrge_capacity = pd.concat(jt_discahrge_capacity_df_list, ignore_index=True)
    before_df_jt_25_soc60_5sec_dr = pd.concat(jt_25_soc60_5sec_dr_df_list, ignore_index=True)
    before_df_jt_25_soc60_02sec_dr = pd.concat(jt_25_soc60_02sec_dr_df_list, ignore_index=True)
    before_df_jt_n10_soc15_5sec_cr = pd.concat(jt_n10_soc15_5sec_cr_df_list, ignore_index=True)
    before_df_jt_n30_soc15_2sec_dr = pd.concat(jt_n30_soc15_2sec_dr_df_list, ignore_index=True)
    before_df_jt_n30_soc15_02sec_dr = pd.concat(jt_n30_soc15_02sec_dr_df_list, ignore_index=True)
    before_df_jt_n10_soc15_5sec_dr = pd.concat(jt_n10_soc15_5sec_dr_df_list, ignore_index=True)
    before_df_jt_n10_soc15_02sec_dr = pd.concat(jt_n10_soc15_02sec_dr_df_list, ignore_index=True)
    
    # JH/JM
    before_df_jh_discharge_capacity = pd.concat(jh_discharge_capacity_df_list, ignore_index=True)
    before_df_jh_25_soc50_10sec = pd.concat(jh_25_soc50_10sec_df_list, ignore_index=True)
    before_df_jh_n35_soc50_10sec = pd.concat(jh_n35_soc50_10sec_df_list, ignore_index=True)
    before_df_jh_25_soc10_10sec = pd.concat(jh_25_soc10_10sec_df_list, ignore_index=True)
    before_df_jm_0_soc70_10sec = pd.concat(jm_0_soc70_10sec_df_list, ignore_index=True)
    
    # 各DFを一つのtupleにまとめる
    before_df_tuple = tuple([before_df_jt_discahrge_capacity, before_df_jt_25_soc60_5sec_dr, before_df_jt_25_soc60_02sec_dr,
                           before_df_jt_n10_soc15_5sec_cr, before_df_jt_n30_soc15_2sec_dr, before_df_jt_n30_soc15_02sec_dr,
                           before_df_jt_n10_soc15_5sec_dr, before_df_jt_n10_soc15_02sec_dr,
                           before_df_jh_discharge_capacity, before_df_jh_25_soc50_10sec, before_df_jh_n35_soc50_10sec,
                           before_df_jh_25_soc10_10sec, before_df_jm_0_soc70_10sec])
    
    # 作成したリストをDFに変換
    df_after = pd.DataFrame(all_ch_result_list, columns=["Date", "ChNo", "DischargeCap(25℃)JT", "DischargeRes(5sec25℃2.5VSoc60%)", "DischargeRes(2sec-30℃2.0VSoc15%)", "DischargeRes(5sec-10℃2.5VSoc15%)", 
                                                "ChargeRes(5sec-10℃4.175VSoc15%)", "DischargeRes(0.2sec25℃2.5VSoc60%)", "DischargeRes(0.2sec-30℃2.0VSoc15%)", "DischargeRes(0.2sec-10℃2.5VSoc15%)",  
                                                "DischargeCap(25℃)JHJM", "DischargeOut(10sec25℃2.5VSoc50%)", "DischargeOut(10sec-35℃2.5VSoc50%)", "DischargeOut(10sec25℃2.5VSoc10%)", "ChargeOut(10sec0℃4.3VSoc70%)",
                                                "DischargeRes(10sec25℃2.5VSoc50%)", "DischargeRes(10sec-35℃2.5VSoc50%)", "DischargeRes(10sec25℃2.5VSoc10%)", "ChargeRes(10sec0℃4.3VSoc70%)"])
    
    
    
    return df_after, before_df_tuple, person

### 各検査項目の結果を判定する関数

##### JT

In [17]:
# 結果判定用の関数
def judge_jt_discahrge_capacity(dc=None):
    if 49.0 <= dc <= 53.0:
        return "OK"
    else:
        return "NG"

def judge_jt_25_soc60_5sec_dr(dr=None):
    if 0.92 <= dr <= 1.07:
        return "OK"
    else:
        return "NG"
    
def judge_jt_25_soc60_02sec_dr(dr=None):
    if 0.65 <= dr <= 0.76:
        return "OK"
    else:
        return "NG"
    

def judge_jt_n30_soc15_2sec_dr(dr=None):
    if 11.13 <= dr <= 15.06:
        return "OK"
    else:
        return "NG"


def judge_jt_n30_soc15_02sec_dr(dr=None):
    if 7.05 <= dr <= 9.54:
        return "OK"
    else:
        return "NG"

    
def judge_jt_n10_soc15_5sec_cr(cr=None):
    if 2.64 <= cr <= 3.57:
        return "OK"
    else:
        return "NG"


def judge_jt_n10_soc15_5sec_dr(dr=None):
    if 5.82 <= dr <= 7.88:
        return "OK"
    else:
        return "NG"

    
def judge_jt_n10_soc15_02sec_dr(dr=None):
    if 2.36 <= dr <= 3.19:
        return "OK"
    else:
        return "NG"

##### JH/JM

In [18]:
# 結果判定用の関数
def judge_jh_discharge_capacity(dc=None):
    if dc >= 50.0:
        return "OK"
    else:
        return "NG"

def judge_jh_25_soc50_10sec_w(w=None):
    if w >= 1053.2:
        return "OK"
    else:
        return "NG"
    
def judge_jh_25_soc50_10sec_dr(dr=None):
    if 0.90 <= dr <= 1.155:
        return "OK"
    else:
        return "NG"
    
def judge_jh_n35_soc50_10sec_w(w=None):
    if w >= 248.9:
        return "OK"
    else:
        return "NG"

def judge_jh_n35_soc50_10sec_dr(dr=None):
    if 6.50 <= dr <= 9.893:
        return "OK"
    else:
        return "NG"


def judge_jh_25_soc10_10sec_w(w=None):
    if w >= 873.3:
        return "OK"
    else:
        return "NG"


def judge_jh_25_soc10_10sec_dr(dr=None):
    if 1.52 <= dr <= 2.364:
        return "OK"
    else:
        return "NG"

    
def judge_jm_0_soc70_10sec_w(w=None):
    if w >= 531.0:
        return "OK"
    else:
        return "NG"

    
def judge_jm_0_soc70_10sec_dr(dr=None):
    if 1.84 <= dr <= 2.36:
        return "OK"
    else:
        return "NG"

### Oracle DB 接続用の関数

In [19]:
#CX oracleでMESサーバーへ接続（接続情報以外は定型）
class conn_MES_LWR:
    def __init__(self, host= "10.60.28.21", port="1521", service="psh1dbv",
                       scheme="tabuser",username="tabuser",password="tab123"):        
        self.host = host
        self.port = port
        self.service  = service

        self.scheme   = scheme
        self.username = username
        self.password = password
    
    def __enter__(self):
        
        # tns:Oracleが命名したDB接続用インターフェース技術の名前
        
        # インターフェイスオブジェクトの作成
        self.tns  = oracle.makedsn(self.host, self.port, service_name=self.service) if self.host else None
        # 接続を確立
        self.conn = oracle.connect(self.username, self.password, self.tns) if self.tns else None
        # カーソルの取得
        self.curs = self.conn.cursor() if self.conn else None
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        if self.curs is not None: self.curs.close()
        if self.conn is not None: self.conn.close()

### DBにデータをアップする関数

In [20]:
def upload_data(insert_sql=None, df=None):
    
    # DBに格納できるようにデータフレームを二次元配列に変換
    rows = [list(x) for x in df.values]
    
    with conn_MES_LWR() as mesdb:
        # executemany()で複数行のデータを一括でインサート
        mesdb.curs.executemany(insert_sql, rows)
        mesdb.conn.commit()

### 各テーブル用のSQLを定義

In [21]:
# 抜取テーブル用SQL
insert_sample = 'INSERT INTO "TABUSER"."ReliabilityTest(Smp)26TA" ("KLot", "LineNo", "CellId", "JigNo", "CycleNo", "ChNo", "RecDate", "InputDate", "TestPtn", "TerminalRes(+)[mΩ]", "TerminalRes(-)[mΩ]", "TotalTerminalRes[mΩ]", "JudgeRes", "Note", "CycleEndDate") VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15)'
# 抜取セル幅寸法テーブル用SQL
insert_cell_size = 'INSERT INTO "TABUSER"."ReliabilityTest(SmpSize)26TA" ("CellId", "CellSize(Top)", "CellSize(Bottom)") VALUES (:1,:2,:3)'
# 演算前テーブル用SQL (JT充電抵抗-10℃SOC15%5sec 14列)
insert_before1 = 'INSERT INTO "TABUSER"."ReliabilityTest(BefCal)26TA" ("TestDate", "ModType", "Klot", "LineNo", "MachineNo", "ChNo", "PtnNo", "EndTime[sec]", "EndCurr[A]", "EndVolt[V]", "InitTemp1[℃]", "InitVolt[V]", "CellId", "Item") VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14)'
# 演算前テーブル用SQL (JT/JH放電容量 15列)
insert_before2 = 'INSERT INTO "TABUSER"."ReliabilityTest(BefCal)26TA" ("TestDate", "ModType", "Klot", "LineNo", "MachineNo", "ChNo", "EndCapA[Ah]", "PtnNo", "EndTime[sec]", "EndCurr[A]", "EndVolt[V]", "EndTemp1[℃]", "InitVolt[V]", "CellId", "Item") VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15)'
# 演算前テーブル用SQL (それ以外 12列)
insert_before3 = 'INSERT INTO "TABUSER"."ReliabilityTest(BefCal)26TA" ("TestDate", "ModType", "Klot", "LineNo", "MachineNo", "ChNo", "StpTime[sec]", "Curr[A]", "Volt[V]", "Temp1[℃]", "CellId", "Item") VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12)'
# 演算後テーブル用SQL
insert_after = 'INSERT INTO "TABUSER"."ReliabilityTest(AftCal)26TA" ("TestDate", "ModType", "Klot", "LineNo", "MachineNo", "ChNo", "DischargeCap(25℃)JT", "DischargeRes(5sec25℃2.5VSoc60%)", "DischargeRes(2sec-30℃2.0VSoc15%)", "DischargeRes(5sec-10℃2.5VSoc15%)", "ChargeRes(5sec-10℃4.175VSoc15%)", "DischargeRes(0.2sec25℃2.5VSoc60%)", "DischargeRes(0.2sec-30℃2.0VSoc15%)", "DischargeRes(0.2sec-10℃2.5VSoc15%)", "DischargeCap(25℃)JHJM", "DischargeOut(10sec25℃2.5VSoc50%)", "DischargeOut(10sec-35℃2.5VSoc50%)", "DischargeOut(10sec25℃2.5VSoc10%)", "ChargeOut(10sec0℃4.3VSoc70%)", "DischargeRes(10sec25℃2.5VSoc50%)", "DischargeRes(10sec-35℃2.5VSoc50%)", "DischargeRes(10sec25℃2.5VSoc10%)", "ChargeRes(10sec0℃4.3VSoc70%)", "CellId", "DischargeCap(25℃)JTResult", "DischargeRes(5sec25℃2.5VSoc60%)Result", "DischargeRes(2sec-30℃2.0VSoc15%)Result", "DischargeRes(5sec-10℃2.5VSoc15%)Result", "ChargeRes(5sec-10℃4.175VSoc15%)Result", "DischargeRes(0.2sec25℃2.5VSoc60%)Result", "DischargeRes(0.2sec-30℃2.0VSoc15%)Result", "DischargeRes(0.2sec-10℃2.5VSoc15%)Result", "DischargeCap(25℃)JHJMResult", "DischargeOut(10sec25℃2.5VSoc50%)Result", "DischargeOut(10sec-35℃2.5VSoc50%)Result", "DischargeOut(10sec25℃2.5VSoc10%)Result", "ChargeOut(10sec0℃4.3VSoc70%)Result", "DischargeRes(10sec25℃2.5VSoc50%)Result", "DischargeRes(10sec-35℃2.5VSoc50%)Result", "DischargeRes(10sec25℃2.5VSoc10%)Result", "ChargeRes(10sec0℃4.3VSoc70%)Result", "FinalResult") VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,:25,:26,:27,:28,:29,:30,:31,:32,:33,:34,:35,:36,:37,:38,:39,:40,:41,:42)'

### セル寸法のDF作成＆DBへのアップロード用関数

In [22]:
def upload_cell_size(csv_file_path=None):
    
    # 試験装置から出力されるcsvファイルを読み込む
    sample_csv_path = csv_file_path
    df_cell_size = pd.read_csv(sample_csv_path, encoding='shift-jis')
    
    # 必要な行のみに絞る('測定時刻'がNaNの行は不要)
    df_cell_size.dropna(subset = ['測定時刻'], inplace=True)
    
    #NaN -> 空文字に変換(NaNのままだとDBにインサートできない)
    df_cell_size.fillna("", inplace=True)
    
    # DB用に必要な列のみに絞る
    df_cell_size_col_filtered = df_cell_size[["シリアル　カウンタ", "[1]底面　3ｍｍ", "[2]天面　3mm"]]
    
    # 抜取セル寸法DFをDBにアップロード
    upload_data(insert_cell_size, df_cell_size_col_filtered)
    
    return df_cell_size_col_filtered

### 抜取用DFの作成・DBへのアップロード用関数

In [23]:
def upload_df_sample(cycleEnd_filePath=None, target_folder_name=None):
    
    # フォルダ名から各検査ロットの設備ライン号機を取得
    line_number_str = re.findall("\(\d\)", target_folder_name)
    # 取得した文字列のindex 1 がライン番号
    line_number = int(line_number_str[0][1])
    
    # Excelのファイル名を設定
    excel_path = "★信頼性試験 管理表 測定データ KZ109~.xlsx"

    # Excelファイルの読み込み (設備ライン番号によって使用するシート名を振り分け)
    if line_number == 1:
        df_sample = pd.read_excel(excel_path, sheet_name="1号_サイクル", header=1).iloc[:, 0:15]
    elif line_number == 2:
        df_sample = pd.read_excel(excel_path, sheet_name="2号＿サイクル", header=1).iloc[:, 0:15]


    # 検査ロットをファイル内のデータから抽出
    with open(cycleEnd_filePath) as f:
        for row, text in enumerate(f, start=0):
            text = text.rstrip()
            if "ロットNo." in text:
                lot_no = text
                break
        else:
            raise ValueError('Not Found')

    # ロットNoをあるだけ取り出す
    res = re.findall("\w{0,3}\d{2,3}", lot_no)

    # ロットNoのリストを作成
    lot_list = []
    for idx, lot in enumerate(res):
        if idx == 0:
            prefix = lot[:2]
        elif len(lot) == 3:
            lot = prefix + lot
        lot_list.append(lot)

    # 対象ロットNoで絞り込み
    df_list = []
    for lot in lot_list:
        if len(df_sample[df_sample["検査LOT"] == lot]):
            df_list.append(df_sample[df_sample["検査LOT"] == lot])

    df_sample = pd.concat(df_list, ignore_index=True)

    # サイクル終了予定日を全行にコピー(全行共通)
    # "サイクル終了予定日"がExcelの15行目
    df_sample["サイクル終了予定日"] = df_sample.iat[0, 14]


    #NaN -> 空文字に変換(NaNのままだとDBにインサートできない)
    df_sample.fillna("", inplace=True)
    
    
    #　抜取用DFをDBにアップロード
    upload_data(insert_sample, df_sample)


    # 演算後のDF用に抜取DFから必要な列を取ってくる
    # モジュール機種:検査ロット:ライン番号:設備号機:チャンネル番号
    df_sample_for_after = df_sample.iloc[:, [8, 0, 1, 2, 4]].copy()


    # サンプルデータフレームの列名を変更
    df_sample_for_after.rename(columns={'試験ﾊﾟﾀｰﾝ':'ModType','検査LOT':'Klot', 'ライン番号':'LineNo', 'セルID':'CellId', '投入サイクル機No':'MachineNo'}, inplace=True)
    
    return df_sample_for_after


### 演算前DFをまとめてDBにアップロードする関数

In [23]:
# 検査項目毎の演算前DFリストの中身
# 0:JT放電容量
# 1:JT 放電抵抗25℃SOC60%5sec
# 2:JT 放電抵抗25℃SOC60%0.2sec
# 3:JT充電抵抗-10℃SOC15%5sec
# 4:JT 放電抵抗-30℃SOC15%2sec
# 5:JT 放電抵抗-30℃SOC15%0.2sec
# 6:JT 放電抵抗-10℃SOC15%5sec
# 7:JT 放電抵抗-10℃SOC15%0.2sec
# 8:JH放電容量
# 9:JH 放電出力・放電抵抗25℃SOC50%10sec
# 10:JH 放電出力・放電抵抗-35℃SOC50%10sec
# 11:JH 放電出力・放電抵抗25℃SOC10%10sec
# 12:JM 充電出力・充電抵抗0℃SOC70%10sec

In [24]:
def upload_df_before(df_before_list=None):
    
    # 全ての検査項目の演算前DFを列数が同じものにグルーピング
    df_dc_list = []
    df_dr_list = []

    for idx, df in enumerate(df_before_list):

        # 0,8は放電量量のDF
        if idx in [0, 8]:
            df_dc_list.append(df)
        # 3はJT充電抵抗-10℃SOC15%5secのDF
        elif idx != 3:
            df_dr_list.append(df)
    
    # グルーピングしたDFを一つのDFにまとめる
    df_dc_for_db = pd.concat(df_dc_list, ignore_index=True)
    df_dr_for_db = pd.concat(df_dr_list, ignore_index=True)
    df_cr_for_db = df_before[3]
    
    
    # 演算前DFをDB用に列を並び替え(JT充電抵抗-10℃SOC15%5sec)
    df_cr_for_db = df_cr_for_db.reindex(columns=["TestDate", "ModType", "Klot", "LineNo", "MachineNo", "ChNo", \
                                     "PtnNo.", "EndTime[sec]", \
                                     "EndCurr[A]", "EndVolt[V]", "InitTemp1[℃]", "InitVolt[V]", "CellId", "Item"])

    # 演算前DFをDB用に列を並び替え(JT/JH放電容量)
    df_dc_for_db = df_dc_for_db.reindex(columns=["TestDate", "ModType", "Klot", "LineNo", "MachineNo", "ChNo", \
                                     "EndCapA[Ah]", "PtnNo.", "EndTime[sec]", \
                                     "EndCurr[A]", "EndVolt[V]", "EndTemp1[℃]", "InitVolt[V]", "CellId", "Item"])

    
    # 演算前DFをDB用に列を並び替え(それ以外)
    df_dr_for_db = df_dr_for_db.reindex(columns=["TestDate", "ModType", "Klot", "LineNo", "MachineNo", "ChNo", \
                                     "StpTime[sec]", "Curr[A]", "Volt[V]", "Temp1[℃]", "CellId", "Item"])
    
    
    upload_data(insert_before1, df_cr_for_db)
    upload_data(insert_before2, df_dc_for_db)
    upload_data(insert_before3, df_dr_for_db)

### 演算後DFをまとめてDBにアップロードする関数

In [25]:
def upload_df_after(df_after=None, df_sample_for_after=None):
    
    
    # 演算後のDFに抜取用DFを挿入
    col_name_list = ["ModType", "Klot", "LineNo", "MachineNo", "CellId"]
    for idx, col_name in enumerate(col_name_list, 1):
        if col_name == 'CellId':
            # セルIDを最終列に挿入
            df_after.insert(loc=23, column=col_name, value=df_sample_for_after[col_name])
        else:
            df_after.insert(loc=idx, column=col_name, value=df_sample_for_after[col_name])
     

    # 各項目の結果判定(各項目毎の列に結果判定用の関数を.apply()してるだけ)
    # JT項目の判定
    df_after["DischargeCap(25℃)JTResult"] = df_after["DischargeCap(25℃)JT"].apply(judge_jt_discahrge_capacity)
    df_after["DischargeRes(5sec25℃2.5VSoc60%)Result"] = df_after["DischargeRes(5sec25℃2.5VSoc60%)"].apply(judge_jt_25_soc60_5sec_dr)
    df_after["DischargeRes(2sec-30℃2.0VSoc15%)Result"] = df_after["DischargeRes(2sec-30℃2.0VSoc15%)"].apply(judge_jt_n30_soc15_2sec_dr)
    df_after["DischargeRes(5sec-10℃2.5VSoc15%)Result"] = df_after["DischargeRes(5sec-10℃2.5VSoc15%)"].apply(judge_jt_n10_soc15_5sec_dr)
    df_after["ChargeRes(5sec-10℃4.175VSoc15%)Result"] = df_after["ChargeRes(5sec-10℃4.175VSoc15%)"].apply(judge_jt_n10_soc15_5sec_cr)
    df_after["DischargeRes(0.2sec25℃2.5VSoc60%)Result"] = df_after["DischargeRes(0.2sec25℃2.5VSoc60%)"].apply(judge_jt_25_soc60_02sec_dr)
    df_after["DischargeRes(0.2sec-30℃2.0VSoc15%)Result"] = df_after["DischargeRes(0.2sec-30℃2.0VSoc15%)"].apply(judge_jt_n30_soc15_02sec_dr)
    df_after["DischargeRes(0.2sec-10℃2.5VSoc15%)Result"] = df_after["DischargeRes(0.2sec-10℃2.5VSoc15%)"].apply(judge_jt_n10_soc15_02sec_dr)

    # JH/JM項目の判定
    df_after["DischargeCap(25℃)JHJMResult"] = df_after["DischargeCap(25℃)JHJM"].apply(judge_jh_discharge_capacity)
    df_after["DischargeOut(10sec25℃2.5VSoc50%)Result"] = df_after["DischargeOut(10sec25℃2.5VSoc50%)"].apply(judge_jh_25_soc50_10sec_w)
    df_after["DischargeOut(10sec-35℃2.5VSoc50%)Result"] = df_after["DischargeOut(10sec-35℃2.5VSoc50%)"].apply(judge_jh_n35_soc50_10sec_w)
    df_after["DischargeOut(10sec25℃2.5VSoc10%)Result"] = df_after["DischargeOut(10sec25℃2.5VSoc10%)"].apply(judge_jh_25_soc10_10sec_w)
    df_after["ChargeOut(10sec0℃4.3VSoc70%)Result"] = df_after["ChargeOut(10sec0℃4.3VSoc70%)"].apply(judge_jm_0_soc70_10sec_w)
    df_after["DischargeRes(10sec25℃2.5VSoc50%)Result"] = df_after["DischargeRes(10sec25℃2.5VSoc50%)"].apply(judge_jh_25_soc50_10sec_dr)
    df_after["DischargeRes(10sec-35℃2.5VSoc50%)Result"] = df_after["DischargeRes(10sec-35℃2.5VSoc50%)"].apply(judge_jh_n35_soc50_10sec_dr)
    df_after["DischargeRes(10sec25℃2.5VSoc10%)Result"] = df_after["DischargeRes(10sec25℃2.5VSoc10%)"].apply(judge_jh_25_soc10_10sec_dr)
    df_after["ChargeRes(10sec0℃4.3VSoc70%)Result"] = df_after["ChargeRes(10sec0℃4.3VSoc70%)"].apply(judge_jm_0_soc70_10sec_dr)

    
    # 総合判定の列を追加
    df_after["FinalResult"] = ""
    
    # 総合判定を実施 (どれか一項目でもNGがあれば総合判定もNG)
    for idx, data in df_after.iterrows():
        judge_list = [judge for judge in data[24:]]
        if any([elem == 'NG' for elem in judge_list]):
            # 41列目が'judge'列
            df_after.iloc[idx, 41] = 'NG'
        else:
            df_after.iloc[idx, 41] = 'OK'
    
    upload_data(insert_after, df_after)
    
    return df_after

### Excel出力用に演算後DFを加工する関数 

In [26]:
def make_df_result(df_after=None):
    
    # 担当者の列を追加
    df_after_to_excel = df_after.copy()
    df_after_to_excel["担当者"] = person
    
    # Excel出力用に必要な列をDBの順序で取得
    df_after_to_excel = df_after_to_excel.iloc[:, [3, 4, 5, 2, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, -1, -2]]
    
    # 寸法天面3mm, 寸法底面3mm の列を追加
    df_after_to_excel.insert(loc=12, column="Top", value=None)
    df_after_to_excel.insert(loc=13, column="Bottom", value=None)
    
    return df_after_to_excel

### Excel出力用に抜取セル寸法のDFを加工する関数 

In [27]:
def make_df_cell_size_to_excel(df_cell_size=None):
    
    # 抜取セル寸法DFをコピー
    df_cell_size_to_excel = df_cell_size.copy()

    # 担当者及び判定の列を追加(元のcsvファイルにはOKとなったものしか入ってないので、"OK"で固定)
    df_cell_size_to_excel["担当者"] = person
    df_cell_size_to_excel["判定"] = "OK"
    
    # ライン番号・検査ロットの列を追加
    df_cell_size_to_excel.insert(loc=0, column="LineNo", value=df_after_to_excel["LineNo"])
    df_cell_size_to_excel.insert(loc=1, column="Klot", value=df_after_to_excel["Klot"])
    
    # DBの列の並びに整頓
    df_cell_size_to_excel = df_cell_size_to_excel.reindex(columns=["LineNo", "Klot", "[1]底面　3ｍｍ", "[2]天面　3mm", "シリアル　カウンタ", "担当者", "判定"])
    
    # ダミー列を追加して列の数をExcel合わせる
    dummy_list_index = [1,2,4,5,6,7,8,9,10,11,14,15,16,17,18,19,20,21]

    for idx in dummy_list_index:
        dummy_col_name = "dummy" + str(idx)
        df_cell_size_to_excel.insert(loc=idx, column=dummy_col_name, value=None)
        
    return df_cell_size_to_excel

### 2次元配列のデータを一括でExcelに書き込む関数

In [28]:
def write_list_2d(sheet, l_2d, start_row, start_col):
    for y, row in enumerate(l_2d):
        for x, cell in enumerate(row):
            sheet.cell(row=start_row + y,
                       column=start_col + x,

                       value=l_2d[y][x])

### Excelに結果を出力する関数

In [52]:
def output_to_excel(df_to_excel=None):

    # カレントディレクトリを取得
    cwd = os.getcwd()
    
    # 結果出力先のExcelファイル名を取得
    for f in glob.glob(cwd + "\\*"):
        file_name = os.path.split(f)[1]
        if re.findall("result", file_name):
            excel_path = file_name
    
    #print(excel_path)
    
    # 最終的に出力するDFを二次元配列に変換
    l_2d = df_to_excel.values.tolist()
    
    # Excelファイルの読み込み
    wb = openpyxl.load_workbook(excel_path)
    # ワークシートの読み込み
    ws = wb['Sheet1']

    # データ追加行を設定
    result_df = pd.read_excel("result.xlsx", sheet_name="Sheet1")
    result_df = result_df.where(result_df.notna(), None)
    
    max_row = len([line_no for line_no in result_df["LotNo."] if isinstance(line_no, str)])
    
    insert_row_num = max_row + 2
    print(insert_row_num)
    write_list_2d(ws, l_2d, insert_row_num, 1)

    # Excelファイルを保存
    wb.save(excel_path)
    
    # Excelを閉じる
    wb.close()

# 複数フォルダ対応の処理

In [30]:
# 必要なcsvファイルをチャネルごとにまとめてリスト化
csv_lists = make_file_path_list()

In [31]:
# フォルダ内にあるセル寸法のcsvファイル名を全て取得してリスト化
cell_size_csv_list = []

# カレントディレクトリを取得
cwd = os.getcwd()

# セル寸法データが入ったcsvファイル名を取得
for f in glob.glob(cwd + "\\*"):
    file_name = os.path.split(f)[1]
    if re.findall("TA承認図", file_name):
        cell_size_csv_list.append(file_name)
        


# 各csvファイルから検査ロット番号を取得してファイル名と紐づける
klot_no_list = []
for cell_size_csv in cell_size_csv_list:
    
    # 検査ロットをファイル内のデータから抽出
    with open(cell_size_csv) as f:
        for row, text in enumerate(f, start=0):
            # 1行目はヘッダなのでスルー
            if row == 0:
                continue

            text = text.rstrip()
            if "1" in text:
                row_string = text
                break
        else:
            raise ValueError('Not Found')
            
    klot_no = re.findall('"(\w+\d+)"', row_string)      
    
    # re.findall()はリストを返すので、そのリストの最初の要素をklot_no_listに追加
    # csvファイル名と検査ロット番号を対応させてタプルに格納
    klot_no_list.append((cell_size_csv, klot_no[0]))

In [54]:
# 各フォルダ毎の演算後DFを格納するリストを準備
after_dfs = []

for folder_name, csv_list in csv_lists:
    
    # セル寸法のcsvファイルと生データフォルダを紐づけ
    for csv_file, klot in klot_no_list:

        if klot in folder_name:
            target_csv_file = csv_file
    
    # csv_list[0]はそのフォルダの開始チャネル番号が入っているので開始インデックスは１～
    for ch_num, ch_folder in enumerate(csv_list[1:], 1):
        
        if ch_folder:
            
            # 開始チャネルのCycleEnd-000k.csvのパスを取得
            cycleEnd_filePath = ch_folder[0]
            break
        
            
    # 抜取用DFの作成＆アップロード
    df_sample_for_after = upload_df_sample(cycleEnd_filePath, folder_name)
    
    # セル寸法のDF作成＆アップロード
    df_cell_size = upload_cell_size(target_csv_file)
    
    # 演算処理および演算前後のＤＦを準備
    df_after, df_before, person = make_result_list(csv_list[1:], df_sample_for_after)
    
    # 演算前DFをDBにアップロード
    upload_df_before(df_before)
    
    # 演算後DFをDBにアップロード＋演算後DFをExcel出力用に加工
    df_after = upload_df_after(df_after, df_sample_for_after)
    # 演算後DFをリストに追加
    after_dfs.append(df_after)
    
    # 演算後DFに'NG'の行があるかどうかをチェック
    ng_data = df_after.query("FinalResult == 'NG'")
    
    # 全ての行が'OK'の時('NG'行がない)のみ結果をExcelに出力
    if not len(ng_data):
        # 演算後DFに担当者・セル寸法等の列を追加
        df_after_to_excel = make_df_result(df_after)

        # 演算結果のデータをExcelに出力
        output_to_excel(df_after_to_excel)

        # セル寸法のDFをExcel出力用に加工
        df_cell_size_to_excel = make_df_cell_size_to_excel(df_cell_size)

        # セル寸法のデータをExcelに出力
        output_to_excel(df_cell_size_to_excel)

14
20


### 演算結果がNGだった場合に対象箇所をExcelに出力

In [33]:
# NGおよびOKだったDFを振り分ける
ng_df_list = []
ok_df_list = []

# "FinalResult"列(最終列)がNGの行を取り出す
for idx, after_df in enumerate(after_dfs):
    ng_data = after_df.query("FinalResult == 'NG'")
    if len(ng_data):
        ng_df_list.append(ng_data)
    else:
        ok_df_list.append(idx)
        
if len(ng_df_list):
    # NGだったDFを一つにまとめる
    ng_df = pd.concat(ng_df_list)

    # 列名を日本語に変更
    ng_df.rename(columns={'Date':'日付','CellId':'セルID', 'Klot':'検査ロット', 'LineNo':'生産ライン', 
                      'MachineNo':'投入サイクル機No', 'ChNo':'チャネル', 'DischargeCap(25℃)JTResult':'放電容量_at25℃_JT',
                      'DischargeRes(5sec25℃2.5VSoc60%)Result':'放電抵抗(5sec)_at25℃_(2.5V)_SOC60%', 'DischargeRes(2sec-30℃2.0VSoc15%)Result':'放電抵抗(2sec)_at-30℃_(2.0V)_SOC15%',
                      'DischargeRes(5sec-10℃2.5VSoc15%)Result':'放電抵抗(5sec)_at-10℃_(2.5V)_SOC15%', 'ChargeRes(5sec-10℃4.175VSoc15%)Result':'充電抵抗(5sec)_at-10℃_(4.175V)_SOC15%',
                      'DischargeRes(0.2sec25℃2.5VSoc60%)Result':'放電抵抗(0.2sec)_at25℃_(2.5V)_SOC60%', 'DischargeRes(0.2sec-30℃2.0VSoc15%)Result':'放電抵抗(0.2sec)_at-30℃_(2.0V)_SOC15%',
                      'DischargeRes(0.2sec-10℃2.5VSoc15%)Result':'放電抵抗(0.2sec)_at-10℃_(2.5V)_SOC15%', 'DischargeCap(25℃)JHJMResult':'放電容量_at25℃_JHJM', 'DischargeOut(10sec25℃2.5VSoc50%)Result':'放電出力(10sec)_at25℃_(2.5V)_SOC50%',
                      'DischargeOut(10sec-35℃2.5VSoc50%)Result':'放電出力(10sec)_at-35℃_(2.5V)_SOC50%', 'DischargeOut(10sec25℃2.5VSoc10%)Result':'放電出力(10sec)_at25℃_(2.5V)_SOC10%', 'ChargeOut(10sec0℃4.3VSoc70%)Result':'充電出力(10sec)_at0℃_(4.3V)_SOC70%',
                      'DischargeRes(10sec25℃2.5VSoc50%)Result':'放電抵抗(10sec)_at25℃_(2.5V)_SOC50%', 'DischargeRes(10sec-35℃2.5VSoc50%)Result':'放電抵抗(10sec)_at-35℃_(2.5V)_SOC50%', 'DischargeRes(10sec25℃2.5VSoc10%)Result':'放電抵抗(10sec)_at25℃_(2.5V)_SOC10%'}, 
                      inplace=True)
 
    # 作業者にメッセージを出力
    print("NGとなった項目があります。対象箇所をExcelで出力します。再測定を実施してください。")
    
    # NGとなったDFをExcelに出力
    ng_df.to_excel('./NG項目.xlsx')

    input("終了するにはどれかキーを押してください")

# 複数フォルダ未対応の処理

### 抜取用DFの作成＆アップロード

In [33]:
df_sample_for_after = upload_df_sample()

TypeError: expected string or bytes-like object

### セル寸法のDF作成＆DBへのアップロード

In [34]:
df_cell_size = upload_cell_size()

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

### 演算の実行および演算前後のDFを取得

In [33]:
#　関数の実行
df_after, df_before, person = make_result_list(csv_lists[0][1:], df_sample_for_after)

### 演算前DFをDBへアップロード

In [39]:
upload_df_before(df_before)

### 演算後DFをDBにアップロード＋加工した演算後DFを取得

In [40]:
df_after = upload_df_after(df_after, df_sample_for_after)

### Excel出力用に演算後DFを加工

In [41]:
df_after_to_excel = make_df_result(df_after)

### 演算結果のデータをExcelに出力

In [42]:
output_to_excel(df_after_to_excel)

### 抜取セル寸法のDFをExcel出力用に加工 

In [43]:
df_cell_size_to_excel = make_df_cell_size_to_excel(df_cell_size)

### 抜取セル寸法のデータをExcelに出力

In [45]:
output_to_excel(df_cell_size_to_excel)